In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
from nltk.corpus import wordnet
syns = wordnet.synsets("program")

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import re
import unicodedata
import sys

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import urllib.request
import urllib.response
import sys
import os, glob
import http.client, urllib
import json
import http.client, urllib.request, urllib.parse, urllib.error, base64

# Replace the accessKey string value with your valid access key.
accessKey = '82e4612615634c398bfd26e7d6327833'
url = 'westcentralus.api.cognitive.microsoft.com'
path = '/text/analytics/v2.0/keyPhrases'

def extract_keywords(body):
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = http.client.HTTPSConnection(url)
    body_req = {'documents':[{'language': 'en', 'id': 1, 'text': body}]}
    body_json = json.dumps(body_req)
    conn.request ("POST", path, body_json, headers)
    response = conn.getresponse ()
    string = response.read().decode('utf-8')
    json_obj = json.loads(string)
    print("json_obj: ", json_obj)
    keyphrases_list = ((json_obj['documents'])[0])['keyPhrases']
    print("keyphrases_list: ", keyphrases_list)
    return keyphrases_list

In [3]:
# medical conditions training data
training_data = []

import csv

# adds every medical condition along with associated info from csv file
with open('Disease Database CSV NEW.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            training_data.append({"class":row[1], "synonyms":row[2], 
                                  "sentence":row[3], "treatments":row[4], "danger level":row[5]})
        line_count += 1

print ("%s total classes in training data" % len(training_data))

for term in training_data:
    print(term["class"] + " | " + term["synonyms"] + " | " + term["sentence"] + 
          " | " + term["treatments"] + " | " + term["danger level"])

14 total classes in training data
Common Cold | Upper Respiratory Tract;Nose and Throat Infection | Hi, I have a runny nose all the time and I don't know what to do about it. I was sniffing all the time in the past few days and it was getting very annoying;One of my nausal was blocked and I couldn't breath properly;My throat hurts after a cold shower last night, I am not sure what to do about it;It feels like there was something in my throat and I've been dry coughing for several days already;I recently developed a condition that requires frequent washroom visits througout the day;I am so tired this morning, every single one of my muscle hurts so much;All my joints are sour since last morning;I feel so cold all the time, I sneezed all the time | Stay hydrated;rest;sooth a sore throat using saltwater gargle;take over the counter cold and cough medications
 | 1
Allergies | Allergic Reaction | After the soccer game yesterday, I found small bumps on my skin and extreme itchiness;When I was

In [4]:
# a table structure to hold the different punctuation used
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

# method to remove punctuations from sentences.
def remove_punctuation(text):
     return text.translate(tbl)

# method to slice strings
def slicer(my_str,sub):
    index=my_str.find(sub)
    if index !=-1 :
         return my_str[index:] 
    else :
         raise Exception('Sub string not found!')

# # find synonyms of all the words in a sentence
# def find_syns(sentence):
#     words = []
#     for word in sentence.split():
#         if word != '':
#             synonyms = []
#             for syn in wordnet.synsets(word):
#                 for l in syn.lemmas():
#                     if l.name() not in synonyms and l.name() != word:
#                         synonyms.append(l.name())
#             word = (word + ' ' + ' '.join(synonyms)).strip()
#             words.append(word)
#     print("SYNONYMS:",words)
#     return words

# # clean all synonyms and words
# def clean_syns(arg):
#     clean_synonyms = []
#     for syn_set in synonyms:
#         syn_set = ' '.join(s for s in syn_set.split() if not any(c.isdigit() for c in s))
#         syn_set = re.sub(r'\d+', '', syn_set)
#         syn_set = syn_set.replace("_"," ")
#         syn_set = remove_punctuation(syn_set)
#         clean_synonyms.append(syn_set)
#     print("CLEAN SYNONYMS: ",clean_synonyms)
#     return clean_synonyms

In [5]:
import string

words = []
words_total = []
classes = []
classes_details = []
documents = []
ignore_words = ['?']

# loop through each description per symptom
for pattern in training_data:
    symptoms = pattern['sentence']
    word_list = symptoms.split(';')
    print("WORD_LIST:",word_list)
    word_list_new = []
    for word in word_list:
        word = remove_punctuation(word)
        word = word.strip().lower()
        word_list_new.append(word)
    print("WORD_LIST_NEW:",word_list_new)
    joined_symps = ' '.join(s for s in word_list_new)
    print("joined_symps:",joined_symps)
    
    # tokenize each word in the description
    w = nltk.word_tokenize(joined_symps)
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, pattern['class']))
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])
        classes_details.append({"class":pattern['class'], "synonyms":pattern['synonyms'],
                                "treatments":pattern['treatments'], "danger level":pattern['danger level']})
    
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents", documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
print ("classes_details", classes_details)

WORD_LIST: ["Hi, I have a runny nose all the time and I don't know what to do about it. I was sniffing all the time in the past few days and it was getting very annoying", "One of my nausal was blocked and I couldn't breath properly", 'My throat hurts after a cold shower last night, I am not sure what to do about it', "It feels like there was something in my throat and I've been dry coughing for several days already", 'I recently developed a condition that requires frequent washroom visits througout the day', 'I am so tired this morning, every single one of my muscle hurts so much', 'All my joints are sour since last morning', 'I feel so cold all the time, I sneezed all the time']
WORD_LIST_NEW: ['hi i have a runny nose all the time and i dont know what to do about it i was sniffing all the time in the past few days and it was getting very annoying', 'one of my nausal was blocked and i couldnt breath properly', 'my throat hurts after a cold shower last night i am not sure what to do ab

In [10]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    print("TOKENIZED WORDS: ",pattern_words)
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    print("pattern_words:",pattern_words)
    print("BAG: ",bag)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

TOKENIZED WORDS:  ['hi', 'i', 'have', 'a', 'runny', 'nose', 'all', 'the', 'time', 'and', 'i', 'dont', 'know', 'what', 'to', 'do', 'about', 'it', 'i', 'was', 'sniffing', 'all', 'the', 'time', 'in', 'the', 'past', 'few', 'days', 'and', 'it', 'was', 'getting', 'very', 'annoying', 'one', 'of', 'my', 'nausal', 'was', 'blocked', 'and', 'i', 'couldnt', 'breath', 'properly', 'my', 'throat', 'hurts', 'after', 'a', 'cold', 'shower', 'last', 'night', 'i', 'am', 'not', 'sure', 'what', 'to', 'do', 'about', 'it', 'it', 'feels', 'like', 'there', 'was', 'something', 'in', 'my', 'throat', 'and', 'ive', 'been', 'dry', 'coughing', 'for', 'several', 'days', 'already', 'i', 'recently', 'developed', 'a', 'condition', 'that', 'requires', 'frequent', 'washroom', 'visits', 'througout', 'the', 'day', 'i', 'am', 'so', 'tired', 'this', 'morning', 'every', 'single', 'one', 'of', 'my', 'muscle', 'hurts', 'so', 'much', 'all', 'my', 'joints', 'are', 'sour', 'since', 'last', 'morning', 'i', 'feel', 'so', 'cold', 'all'

In [7]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 200)
net = tflearn.fully_connected(net, 100)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=10000, batch_size=16, show_metric=True)
model.save('model.tflearn')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
---------------------------------
Run id: HH81YL
Log directory: tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 14
Validation samples: 0
--
Training Step: 1  | time: 0.140s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 14/14
--
Training Step: 2  | total loss: 2.37620 | time: 0.004s
| Adam | epoch: 002 | loss: 2.37620 - acc: 0.1286 -- iter: 14/14
--
Training Step: 3  | total loss: 2.58921 | time: 0.008s
| Adam | epoch: 003 | loss: 2.58921 - acc: 0.1987 -- iter: 14/14
--
Training Step: 4  | total loss: 2.61009 | time: 0.004s
| Adam | epoch: 004 | loss: 2.61009 - acc: 0.2104 -- iter: 14/14
--
Training Step: 5  | total loss: 2.59774 | time: 0.008s
| Adam | epoch: 005 | loss: 2.59774 - acc: 0.2131 -- iter: 14/14
--
Training Step: 6  | total loss: 2.62243 | time: 0.004s
| Adam | epoch: 006 | loss: 2.622

Training Step: 56  | total loss: 2.90957 | time: 0.008s
| Adam | epoch: 056 | loss: 2.90957 - acc: 0.7318 -- iter: 14/14
--
Training Step: 57  | total loss: 2.50919 | time: 0.004s
| Adam | epoch: 057 | loss: 2.50919 - acc: 0.7689 -- iter: 14/14
--
Training Step: 58  | total loss: 2.17014 | time: 0.004s
| Adam | epoch: 058 | loss: 2.17014 - acc: 0.8004 -- iter: 14/14
--
Training Step: 59  | total loss: 1.88236 | time: 0.004s
| Adam | epoch: 059 | loss: 1.88236 - acc: 0.8272 -- iter: 14/14
--
Training Step: 60  | total loss: 1.63752 | time: 0.004s
| Adam | epoch: 060 | loss: 1.63752 - acc: 0.8501 -- iter: 14/14
--
Training Step: 61  | total loss: 1.42870 | time: 0.004s
| Adam | epoch: 061 | loss: 1.42870 - acc: 0.8697 -- iter: 14/14
--
Training Step: 62  | total loss: 1.25011 | time: 0.004s
| Adam | epoch: 062 | loss: 1.25011 - acc: 0.8864 -- iter: 14/14
--
Training Step: 63  | total loss: 1.09695 | time: 0.004s
| Adam | epoch: 063 | loss: 1.09695 - acc: 0.9008 -- iter: 14/14
--
Training

Training Step: 115  | total loss: 1.45332 | time: 0.004s
| Adam | epoch: 115 | loss: 1.45332 - acc: 0.7219 -- iter: 14/14
--
Training Step: 116  | total loss: 1.34141 | time: 0.008s
| Adam | epoch: 116 | loss: 1.34141 - acc: 0.7497 -- iter: 14/14
--
Training Step: 117  | total loss: 1.24039 | time: 0.004s
| Adam | epoch: 117 | loss: 1.24039 - acc: 0.7747 -- iter: 14/14
--
Training Step: 118  | total loss: 1.54860 | time: 0.004s
| Adam | epoch: 118 | loss: 1.54860 - acc: 0.6973 -- iter: 14/14
--
Training Step: 119  | total loss: 1.42692 | time: 0.004s
| Adam | epoch: 119 | loss: 1.42692 - acc: 0.7275 -- iter: 14/14
--
Training Step: 120  | total loss: 1.31768 | time: 0.004s
| Adam | epoch: 120 | loss: 1.31768 - acc: 0.7548 -- iter: 14/14
--
Training Step: 121  | total loss: 1.21911 | time: 0.004s
| Adam | epoch: 121 | loss: 1.21911 - acc: 0.7793 -- iter: 14/14
--
Training Step: 122  | total loss: 1.48658 | time: 0.004s
| Adam | epoch: 122 | loss: 1.48658 - acc: 0.7157 -- iter: 14/14
--


Training Step: 173  | total loss: 1.28525 | time: 0.004s
| Adam | epoch: 173 | loss: 1.28525 - acc: 0.7466 -- iter: 14/14
--
Training Step: 174  | total loss: 1.48701 | time: 0.008s
| Adam | epoch: 174 | loss: 1.48701 - acc: 0.6791 -- iter: 14/14
--
Training Step: 175  | total loss: 1.37648 | time: 0.004s
| Adam | epoch: 175 | loss: 1.37648 - acc: 0.7112 -- iter: 14/14
--
Training Step: 176  | total loss: 1.55051 | time: 0.004s
| Adam | epoch: 176 | loss: 1.55051 - acc: 0.6615 -- iter: 14/14
--
Training Step: 177  | total loss: 1.43273 | time: 0.008s
| Adam | epoch: 177 | loss: 1.43273 - acc: 0.6953 -- iter: 14/14
--
Training Step: 178  | total loss: 1.65322 | time: 0.004s
| Adam | epoch: 178 | loss: 1.65322 - acc: 0.6330 -- iter: 14/14
--
Training Step: 179  | total loss: 1.52550 | time: 0.004s
| Adam | epoch: 179 | loss: 1.52550 - acc: 0.6697 -- iter: 14/14
--
Training Step: 180  | total loss: 1.76760 | time: 0.008s
| Adam | epoch: 180 | loss: 1.76760 - acc: 0.6027 -- iter: 14/14
--


Training Step: 231  | total loss: 1.22155 | time: 0.004s
| Adam | epoch: 231 | loss: 1.22155 - acc: 0.7650 -- iter: 14/14
--
Training Step: 232  | total loss: 1.12900 | time: 0.008s
| Adam | epoch: 232 | loss: 1.12900 - acc: 0.7885 -- iter: 14/14
--
Training Step: 233  | total loss: 1.04485 | time: 0.004s
| Adam | epoch: 233 | loss: 1.04485 - acc: 0.8097 -- iter: 14/14
--
Training Step: 234  | total loss: 0.96799 | time: 0.004s
| Adam | epoch: 234 | loss: 0.96799 - acc: 0.8287 -- iter: 14/14
--
Training Step: 235  | total loss: 0.89751 | time: 0.008s
| Adam | epoch: 235 | loss: 0.89751 - acc: 0.8458 -- iter: 14/14
--
Training Step: 236  | total loss: 0.83261 | time: 0.004s
| Adam | epoch: 236 | loss: 0.83261 - acc: 0.8613 -- iter: 14/14
--
Training Step: 237  | total loss: 0.77265 | time: 0.024s
| Adam | epoch: 237 | loss: 0.77265 - acc: 0.8751 -- iter: 14/14
--
Training Step: 238  | total loss: 1.08128 | time: 0.004s
| Adam | epoch: 238 | loss: 1.08128 - acc: 0.8019 -- iter: 14/14
--


...

Training Step: 9764  | total loss: 0.98160 | time: 0.004s
| Adam | epoch: 9764 | loss: 0.98160 - acc: 0.8184 -- iter: 14/14
--
Training Step: 9765  | total loss: 0.90753 | time: 0.004s
| Adam | epoch: 9765 | loss: 0.90753 - acc: 0.8366 -- iter: 14/14
--
Training Step: 9766  | total loss: 1.23325 | time: 0.004s
| Adam | epoch: 9766 | loss: 1.23325 - acc: 0.7529 -- iter: 14/14
--
Training Step: 9767  | total loss: 1.13317 | time: 0.004s
| Adam | epoch: 9767 | loss: 1.13317 - acc: 0.7776 -- iter: 14/14
--
Training Step: 9768  | total loss: 1.04291 | time: 0.008s
| Adam | epoch: 9768 | loss: 1.04291 - acc: 0.7999 -- iter: 14/14
--
Training Step: 9769  | total loss: 0.96132 | time: 0.004s
| Adam | epoch: 9769 | loss: 0.96132 - acc: 0.8199 -- iter: 14/14
--
Training Step: 9770  | total loss: 1.23397 | time: 0.004s
| Adam | epoch: 9770 | loss: 1.23397 - acc: 0.7522 -- iter: 14/14
--
Training Step: 9771  | total loss: 1.13292 | time: 0.004s
| Adam | epoch: 9771 | loss: 1.13292 - acc: 0.7770 --

Training Step: 9821  | total loss: 0.83151 | time: 0.004s
| Adam | epoch: 9821 | loss: 0.83151 - acc: 0.8772 -- iter: 14/14
--
Training Step: 9822  | total loss: 0.78112 | time: 0.004s
| Adam | epoch: 9822 | loss: 0.78112 - acc: 0.8894 -- iter: 14/14
--
Training Step: 9823  | total loss: 0.73339 | time: 0.004s
| Adam | epoch: 9823 | loss: 0.73339 - acc: 0.9005 -- iter: 14/14
--
Training Step: 9824  | total loss: 0.68813 | time: 0.004s
| Adam | epoch: 9824 | loss: 0.68813 - acc: 0.9104 -- iter: 14/14
--
Training Step: 9825  | total loss: 0.64519 | time: 0.004s
| Adam | epoch: 9825 | loss: 0.64519 - acc: 0.9194 -- iter: 14/14
--
Training Step: 9826  | total loss: 0.98993 | time: 0.004s
| Adam | epoch: 9826 | loss: 0.98993 - acc: 0.8275 -- iter: 14/14
--
Training Step: 9827  | total loss: 0.91369 | time: 0.004s
| Adam | epoch: 9827 | loss: 0.91369 - acc: 0.8447 -- iter: 14/14
--
Training Step: 9828  | total loss: 1.24875 | time: 0.004s
| Adam | epoch: 9828 | loss: 1.24875 - acc: 0.7602 --

Training Step: 9878  | total loss: 1.83753 | time: 0.012s
| Adam | epoch: 9878 | loss: 1.83753 - acc: 0.5904 -- iter: 14/14
--
Training Step: 9879  | total loss: 1.69187 | time: 0.008s
| Adam | epoch: 9879 | loss: 1.69187 - acc: 0.6313 -- iter: 14/14
--
Training Step: 9880  | total loss: 1.56218 | time: 0.004s
| Adam | epoch: 9880 | loss: 1.56218 - acc: 0.6682 -- iter: 14/14
--
Training Step: 9881  | total loss: 1.44628 | time: 0.028s
| Adam | epoch: 9881 | loss: 1.44628 - acc: 0.7014 -- iter: 14/14
--
Training Step: 9882  | total loss: 1.65222 | time: 0.004s
| Adam | epoch: 9882 | loss: 1.65222 - acc: 0.6384 -- iter: 14/14
--
Training Step: 9883  | total loss: 1.52864 | time: 0.004s
| Adam | epoch: 9883 | loss: 1.52864 - acc: 0.6745 -- iter: 14/14
--
Training Step: 9884  | total loss: 1.41789 | time: 0.004s
| Adam | epoch: 9884 | loss: 1.41789 - acc: 0.7071 -- iter: 14/14
--
Training Step: 9885  | total loss: 1.31817 | time: 0.004s
| Adam | epoch: 9885 | loss: 1.31817 - acc: 0.7364 --

Training Step: 9935  | total loss: 1.38614 | time: 0.004s
| Adam | epoch: 9935 | loss: 1.38614 - acc: 0.7107 -- iter: 14/14
--
Training Step: 9936  | total loss: 1.57756 | time: 0.004s
| Adam | epoch: 9936 | loss: 1.57756 - acc: 0.6540 -- iter: 14/14
--
Training Step: 9937  | total loss: 1.46404 | time: 0.004s
| Adam | epoch: 9937 | loss: 1.46404 - acc: 0.6886 -- iter: 14/14
--
Training Step: 9938  | total loss: 1.65390 | time: 0.004s
| Adam | epoch: 9938 | loss: 1.65390 - acc: 0.6268 -- iter: 14/14
--
Training Step: 9939  | total loss: 1.53335 | time: 0.004s
| Adam | epoch: 9939 | loss: 1.53335 - acc: 0.6642 -- iter: 14/14
--
Training Step: 9940  | total loss: 1.75704 | time: 0.004s
| Adam | epoch: 9940 | loss: 1.75704 - acc: 0.5977 -- iter: 14/14
--
Training Step: 9941  | total loss: 1.62722 | time: 0.004s
| Adam | epoch: 9941 | loss: 1.62722 - acc: 0.6380 -- iter: 14/14
--
Training Step: 9942  | total loss: 1.51076 | time: 0.008s
| Adam | epoch: 9942 | loss: 1.51076 - acc: 0.6742 --

Training Step: 9992  | total loss: 0.67083 | time: 0.004s
| Adam | epoch: 9992 | loss: 0.67083 - acc: 0.8965 -- iter: 14/14
--
Training Step: 9993  | total loss: 0.62660 | time: 0.004s
| Adam | epoch: 9993 | loss: 0.62660 - acc: 0.9068 -- iter: 14/14
--
Training Step: 9994  | total loss: 0.87294 | time: 0.004s
| Adam | epoch: 9994 | loss: 0.87294 - acc: 0.8447 -- iter: 14/14
--
Training Step: 9995  | total loss: 0.80694 | time: 0.004s
| Adam | epoch: 9995 | loss: 0.80694 - acc: 0.8602 -- iter: 14/14
--
Training Step: 9996  | total loss: 1.17646 | time: 0.004s
| Adam | epoch: 9996 | loss: 1.17646 - acc: 0.7742 -- iter: 14/14
--
Training Step: 9997  | total loss: 1.07977 | time: 0.004s
| Adam | epoch: 9997 | loss: 1.07977 - acc: 0.7968 -- iter: 14/14
--
Training Step: 9998  | total loss: 1.34467 | time: 0.008s
| Adam | epoch: 9998 | loss: 1.34467 - acc: 0.7314 -- iter: 14/14
--
Training Step: 9999  | total loss: 1.23184 | time: 0.004s
| Adam | epoch: 9999 | loss: 1.23184 - acc: 0.7583 --

In [8]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y, 'classes_details':classes_details}, open( "training_data", "wb" ) )

In [ ]:
coreml_model = coremltools.converters.keras.convert(model)
coreml_model.save('my_model.mlmodel')